In [6]:
import tensorflow as tf

> A TensorFlow variable is the best way to represent shared, persistent state manipulated by your program.

> Variables are manipulated via the tf.Variable class. A tf.Variable represents a tensor whose value can be changed by running ops on it. Unlike tf.Tensor objects, a tf.Variable exists outside the context of a single session.run call.

> Internally, a tf.Variable stores a persistent tensor. Specific ops allow you to read and modify the values of this tensor. These modifications are visible across multiple tf.Sessions, so multiple workers can see the same values for a tf.Variable.

### Creating variable

>The best way to create a variable is to call the tf.get_variable function. This function requires you to specify the Variable's name. This name will be used by other replicas to access the same variable, as well as to name this variable's value when checkpointing and exporting models. tf.get_variable also allows you to reuse a previously created variable of the same name, making it easy to define models which reuse layers.

In [7]:
'''
    Creates a variable named "my_variable" which is a three-dimensional tensor with shape [1, 2, 3].
    This variable will, by default, have the dtype tf.float32 and 
    its initial value will be randomized via tf.glorot_uniform_initializer.
'''

my_variable = tf.get_variable("my_variable", [1, 2, 3])
print my_variable


'''
    Can change the dtype and initializer of the variable
    
    Initializer can be a tf.Tensor, 
    you should not specify the variable's shape, as the shape of the initializer tensor will be used.
'''
my_int_variable = tf.get_variable("my_int_variable", [1, 2, 3], dtype=tf.int32, initializer=tf.zeros_initializer)
print my_int_variable


other_variable = tf.get_variable("other_variable", dtype=tf.int32, initializer=tf.constant([23, 42]))
print other_variable


In [8]:
'''
    It will raise an error if you create the same-name variable
'''

try:
    my_variable = tf.get_variable("my_variable", [1, 2, 3])
except Exception, e:
    print e
    print

'''
    Use tf.reset_default_graph to reset all declared variable, graph, ... 
'''
tf.reset_default_graph()
my_variable = tf.get_variable("my_variable", [1, 2, 3])
print my_variable

Variable my_variable already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-7-ecad63f6590a>", line 7, in <module>
    my_variable = tf.get_variable("my_variable", [1, 2, 3])
  File "/Users/tranpham/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2885, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/Users/tranpham/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2825, in run_ast_nodes
    if self.run_code(code, result):




### Variable collections

> **tf.GraphKeys.GLOBAL_VARIABLES** --- variables that can be shared across multiple devices, *

> **tf.GraphKeys.TRAINABLE_VARIABLES** --- variables for which TensorFlow will calculate gradients.

In [9]:
'''
    2 ways to create the variable that is not trainable:
        - put to the LOCAL_VARIABLES collection
        - or set trainable = False
'''

my_local = tf.get_variable("my_local", shape=(), collections=[tf.GraphKeys.LOCAL_VARIABLES])
print my_local

my_non_trainable = tf.get_variable("my_non_trainable", shape=(), trainable=False)
print my_non_trainable

In [10]:
'''
    Make a new collection and add the variable to that collection
'''

tf.add_to_collection("my_collection_name", my_local)


In [11]:
'''
    View the list of variables in each collection
'''

print tf.get_collection("my_collection_name")
print tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
print tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES)
print

'''
    GOBAL_VARIABLES seems not in this tensorflow version. Some raise that they got this issue with v11 too. 
'''
try: 
    print tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
except Exception, e:
    print e
    print

[<tensorflow.python.ops.variables.Variable object at 0x116591190>, <tensorflow.python.ops.variables.Variable object at 0x1165478d0>]

type object 'GraphKeys' has no attribute 'GLOBAL_VARIABLES'



### Devices

The variable can also be put on the particular device

> It is particularly important for variables to be in the correct device in distributed settings. Accidentally putting variables on workers instead of parameter servers, for example, can severely slow down training or, in the worst case, let each worker blithely forge ahead with its own independent copy of each variable.

In [12]:
with tf.device("/device:GPU:1"):
    v = tf.get_variable("v", [1])

In [13]:
cluster_spec = {
    "ps": ["ps0:2222", "ps1:2222"],
    "worker": ["worker0:2222", "worker1:2222", "worker2:2222"]}


'''
     Use tf.train.replica_device_setter to automatically place variables in parameter servers
'''
with tf.device(tf.train.replica_device_setter(cluster=cluster_spec)):
    v2 = tf.get_variable("v2", shape=[20, 20])  



### Initialize variable 

> Before you can use a variable, it must be initialized. If you are programming in the low-level TensorFlow API (that is, you are explicitly creating your own graphs and sessions), you must explicitly initialize the variables. Most high-level frameworks such as tf.contrib.slim, tf.estimator.Estimator and Keras automatically initialize variables for you before training a model.

In [19]:
'''
    TensorFlow document introduces the "tf.global_variables_initializer()" method to initialize all global variables
    however, that method is not available in my tensorflow version
    
    After getting the initializer, use session.run() to initialize the variables
'''

all_variables_initializer = tf.initialize_all_variables()


'''
    get many issues with the current tensorflow version
    the below initializer methods also raise argument error
     
    v = tf.get_variable("v", shape=(), initializer=tf.zeros_initializer())
    w = tf.get_variable("w", initializer=v.initialized_value() + 1)

'''


In [36]:
'''
    To assign a value to a variable, use the methods assign, assign_add, and friends in the tf.Variable class. 
    For example, here is how you can call these methods:
'''

v = tf.get_variable("v4", shape=())
assignment = v.assign_add(1)

### Sharing variables / Variable scopes

> Allow you to control variable reuse when calling functions which implicitly create and use variables

> Allow you to name your variables in a hierarchical and understandable way.

In [38]:
def conv_relu(input, kernel_shape, bias_shape):
    # Create variable named "weights".
    weights = tf.get_variable("weights", kernel_shape,
        initializer=tf.random_normal_initializer())
    # Create variable named "biases".
    biases = tf.get_variable("biases", bias_shape,
        initializer=tf.constant_initializer(0.0))
    conv = tf.nn.conv2d(input, weights,
        strides=[1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(conv + biases)


In [39]:
'''
    Cannot call the conv_relu again because "weights" and "biases" were defined
'''

input1 = tf.random_normal([1,10,10,32])
input2 = tf.random_normal([1,20,20,32])
x = conv_relu(input1, kernel_shape=[5, 5, 32, 32], bias_shape=[32])

try:
    x = conv_relu(x, kernel_shape=[5, 5, 32, 32], bias_shape = [32])  # This fails.
except Exception, e:
    print e


Variable weights already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-38-a16efc968852>", line 4, in conv_relu
    initializer=tf.random_normal_initializer())
  File "<ipython-input-39-199793ada2fc>", line 3, in <module>
    x = conv_relu(input1, kernel_shape=[5, 5, 32, 32], bias_shape=[32])
  File "/Users/tranpham/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2885, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)



In [40]:
'''
    Use variable_scope to create new variable (with new name)
'''

def my_image_filter(input_images):
    with tf.variable_scope("conv1"):
        # Variables created here will be named "conv1/weights", "conv1/biases".
        relu1 = conv_relu(input_images, [5, 5, 32, 32], [32])
    with tf.variable_scope("conv2"):
        # Variables created here will be named "conv2/weights", "conv2/biases".
        return conv_relu(relu1, [5, 5, 32, 32], [32])

my_image_filter(x)

<tf.Tensor 'conv2/Relu:0' shape=(1, 10, 10, 32) dtype=float32>

In [42]:

'''
    2 ways to use resued to share the variable
'''

with tf.variable_scope("model2"):
    output1 = my_image_filter(input1)
with tf.variable_scope("model2", reuse=True):
    output2 = my_image_filter(input2)
        
        
with tf.variable_scope("model3") as scope:
    output1 = my_image_filter(input1)
    scope.reuse_variables()
    output2 = my_image_filter(input2)


    
'''
   For the #1 way, beside using the "model" string name, you can also define the scope and reuse that scope 
'''
with tf.variable_scope("model4") as scope:
    output1 = my_image_filter(input1)
with tf.variable_scope(scope, reuse=True):
    output2 = my_image_filter(input2)


